In [21]:
import spacy
import numpy as np
import pandas as pd
import sklearn
import spacy
nlp = spacy.load("en_core_web_sm")

c:\Users\nandi.LAPTOP-6NVS0PRF\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [22]:
doc=pd.read_csv("data2.csv")
print(doc['reply'])

0                                      Weather Chat-bot
1                                    Where do you live?
2               I can only assist you with weather data
3     You can call me weather chat-bot or any other ...
4     I am weather Chat bot, here to tell the curren...
5           Yes if it rain you should take the umbrella
6      If it is sunny and you have to walk a good am...
7     Yes, it is absolutely okay to go out in rain, ...
8      "chat-gpt is a very interesting model intelli...
9     your life is in the hands of Lord Krishna, so ...
10    Everyday morning get a new life, and you can c...
11              That's kinda cute, good morning you too
12    Bro I am a robot, that too very primitive, how...
13                                  Weather info needed
14                                    Good night friend
Name: reply, dtype: object


In [23]:

from sklearn.model_selection import train_test_split
x=doc['message']
y=doc['reply']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.22,random_state=42)



In [24]:
print(y)

0                                      Weather Chat-bot
1                                    Where do you live?
2               I can only assist you with weather data
3     You can call me weather chat-bot or any other ...
4     I am weather Chat bot, here to tell the curren...
5           Yes if it rain you should take the umbrella
6      If it is sunny and you have to walk a good am...
7     Yes, it is absolutely okay to go out in rain, ...
8      "chat-gpt is a very interesting model intelli...
9     your life is in the hands of Lord Krishna, so ...
10    Everyday morning get a new life, and you can c...
11              That's kinda cute, good morning you too
12    Bro I am a robot, that too very primitive, how...
13                                  Weather info needed
14                                    Good night friend
Name: reply, dtype: object


In [25]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),])

text_classifier=Pipeline([('tfidf',TfidfVectorizer()),('rgr',LinearSVC())])
text_classifier.fit(x,y)
text_clf_nb.fit(x,y)



c:\Users\nandi.LAPTOP-6NVS0PRF\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
print(text_classifier.predict(["am i gay?"]))

['Bro I am a robot, that too very primitive, how can i be gay?']


In [27]:
y_pred=text_classifier.predict(x_test)
print(y_pred)
print(sklearn.metrics.accuracy_score(y_test,y_pred))

["your life is in the hands of Lord Krishna, so don't worry, instead live it"
 "That's kinda cute, good morning you too" 'Weather Chat-bot'
 'Weather info needed']
1.0


In [28]:
print(text_classifier.predict(["Shall i take an umbrella with me if it is sunny?"]))

[' If it is sunny and you have to walk a good amount of distance then taking then taking umbrella is preferable.']


In [30]:
from spacy.matcher import PhraseMatcher
def get_query_states(doc):
    doc=nlp(doc)
    matcher=PhraseMatcher(nlp.vocab)
    phrase_list=['Chennai','chennai','mumbai','Kolkata','kolkata','Patna',"punjab","haryana","andhra pradesh","west Bengal"]
    phrase_patterns=[nlp(text) for text in phrase_list]
    matcher.add('states',phrase_patterns)
    matches =matcher(doc)
    query_states=[]
    for match in matches:
        query_states.append(doc[match[1]:match[2]])
        print(doc[match[1]:match[2]])
    return query_states
temp=get_query_states("kolkata")
print(temp)

kolkata
[kolkata]


In [ ]:
import requests
import json
def get_weather_info(state):
    city=state
    # print("query city"+city)
    url="https://api.openweathermap.org/data/2.5/weather?q="+city+"&appid=2683c9ffea4178dd21b56297878601ee"
    raw_data=requests.get(url)
    # print(raw_data)
    data=json.loads(raw_data.text)#Converting string to dictionary
    current_temp=data["main"]["temp"]-273
    return current_temp



In [ ]:
questions=[]
while True:
    qs=[]
    user_input=input("Enter your text:\t")
    qs.append(user_input)
    questions.append(user_input)
    if user_input in questions:
        print("Previously asked!")
        query_states=get_query_states(user_input)  
        if text_classifier.predict(qs)=="Weather info needed" or query_states:
            for state in query_states:
                state=str(state)
                current_temp=get_weather_info(state)
                print("The current temperature in ,",str(state)," is ",str(current_temp)," degree celcius")
        print(text_classifier.predict(qs))

Previously asked!
kolkata
The current temperature in , kolkata  is  20.120000000000005  degree celcius
['Weather info needed']
Previously asked!
['Where do you live?']
Previously asked!
['Bro I am a robot, that too very primitive, how can i be gay?']
Previously asked!
['Everyday morning get a new life, and you can choose what you want to become.']
Previously asked!
['Where do you live?']
Previously asked!
['Where do you live?']


KeyboardInterrupt: Interrupted by user